In [231]:
# import packages
import pandas as pd
import os
import glob

# install packages 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from statsmodels.tsa.api import SimpleExpSmoothing, ExponentialSmoothing, Holt
import warnings
warnings.filterwarnings("ignore")


plt.style.use('seaborn')
plt.rcParams["figure.figsize"] = (16, 8)

In [232]:
def permutation(df):
    # third_level = [x for x in df.columns if x. == 2]
    # second_level = [x for x in df.columns if x.count('_') == 1]
    # first_level = [x for x in df.columns if (x.count('_') == 0 and x!= "total")]
    if ('total' in df.columns):
        df = df.drop('total', axis = 1)
    perm_lst = []
    column_name_lst = []
    ele = df.columns[1]
    if ele.count('_') == 2:
        for x in range(0, len(df.columns)):
            # break the string
            race_x = df.columns[x].split('_')[0]
            gender_x = df.columns[x].split("_")[1]
            generation_x = df.columns[x].split("_")[2]
        # find the element that doesnt belong to the same group (only one character is diff with others)
            for y in range(x+1, len(df.columns)):
                if df.columns[y]!=df.columns[x] :
                    race_y = df.columns[y].split('_')[0]
                    gender_y = df.columns[y].split("_")[1]
                    generation_y = df.columns[y].split("_")[2]
                    if( (race_x == race_y and gender_x  == gender_y and generation_x != generation_y)
                    or (race_x == race_y and gender_x  != gender_y and generation_x == generation_y)
                    or(race_x != race_y and gender_x  == gender_y and generation_x == generation_y)):
                    
                        perm_lst.append([df.columns[x],df.columns[y]])
                        column_name_lst.append(df.columns[x]+" "+df.columns[y])
                    
    elif df.columns[0].count('_') == 1:
        for x in range(0,len(df.columns)):
            # break the string
            race_x = df.columns[x].split('_')[0]
            gender_x = df.columns[x].split("_")[1]
            # find the element that doesnt belong to the same group (only one character is diff with others)
            for y in range(x+1, len(df.columns)):
                if y!=x :
                    race_y = df.columns[y].split('_')[0]
                    gender_y = df.columns[y].split("_")[1]
                    if( (race_x == race_y and gender_x  != gender_y)
                    or (race_x != race_y and gender_x  == gender_y)):
                        perm_lst.append([df.columns[x],df.columns[y]])
                        column_name_lst.append(df.columns[x]+" "+df.columns[y])
                        
    elif df.columns[0].count('_') == 0:
        for x in range(0,len(df.columns)):
            for y in range(x+1,len(df.columns)):
                if df.columns[y] != df.columns[x] and df.columns[y]!= 'total' and df.columns[x]!='total':
                    perm_lst.append([df.columns[x],df.columns[y]])
                    column_name_lst.append(df.columns[x]+" "+df.columns[y])
                    
    return perm_lst, column_name_lst



####################################### Accuracy

def RMSE (df_actual, df_pred):
    # create a dataframe to store all the RMSE
    df_rmse = pd.DataFrame(columns= df_pred.columns, index=range(1))
    # total rmse for all features 
    total_rmse = 0
    # get rmse for each col
    for col in df_pred.columns:
        rmse = sqrt(mean_squared_error(df_actual[col], df_pred[col]))
        df_rmse[col].iloc[0] = rmse
        total_rmse = total_rmse + rmse
    avg_rmse =  total_rmse/len(df_pred.columns)
    num_of_cols = len(df_pred.columns)
    return df_rmse, total_rmse, avg_rmse, num_of_cols

def MASE(df_actual, df_pred, df_naive):
    # create a dataframe to store all the MASE
    df_mase = pd.DataFrame(columns= df_pred.columns, index=range(1))
    # total rmse for all features 
    total_mase = 0
    # get rmse for each col
    for col in df_pred.columns:
        # get the MAE_naive
        mae_naive = 0
        for i in range(0, len(df_naive[col])-1):
            abs_diff = abs(df_naive[col].iloc[i+1]-df_naive[col].iloc[i])
            mae_naive = mae_naive + abs_diff
    
        mae_naive = mae_naive/(len(df_naive[col])-1)
        
        mae_pred = mean_absolute_error(df_actual[col], df_pred[col])
        mase = mae_pred/mae_naive
        df_mase[col].iloc[0] = mase
        total_mase = total_mase + mase
        
        avg_mase =  total_mase/len(df_pred.columns)
        num_of_cols = len(df_pred.columns)
    return df_mase, total_mase, avg_mase, num_of_cols



####################################### Fairness

def total_fairness(df_actual, df_pred, thred, level):
 
    if level == "l1":
        diffs = []
        count_actual = 0
    
        actual_total = df_actual['total'].iloc[0]
        columns = [x for x in df_actual.columns if (x.count("_") == 0) and x != 'total']
        perm_lst, column_name_lst = permutation(df_actual[columns])
        for x in perm_lst:
            if (abs(df_actual[x[0]].iloc[0]/actual_total - df_actual[x[1]].iloc[0]/actual_total)) <= thred:
                count_actual = count_actual + 1
        totalfair_actual = count_actual/36

        count_pred = 0
        totalfair_pred = 0
        pred_total = df_pred['total'].iloc[0]
        columns = [x for x in df_pred.columns if (x.count("_") == 0) and x != 'total']
        perm_lst, column_name_lst = permutation(df_pred[columns])
        for x in perm_lst:
            diff = abs(df_pred[x[0]].iloc[0]/pred_total - df_pred[x[1]].iloc[0]/pred_total)
            diffs.append(diff)
            if (diff) <= thred:
                count_pred = count_pred + 1
        totalfair_pred = count_pred/36
        return totalfair_actual, totalfair_pred, diffs
    
    elif level == "l2":
        diffs = []

        count_actual = 0
        totalfair_actual = 0
        race_columns = [x for x in df_actual.columns if (x.count("_") == 0) and x != 'total']
        perm_lst, column_name_lst = permutation(df_actual[race_columns])
        for x in perm_lst:
            fem1 = x[0] + '_'+'fem'
            fem2 = x[1] + "_"+'fem'
            diff = abs((df_actual[fem1].iloc[0]/df_actual[x[0]].iloc[0]) - (df_actual[fem2].iloc[0]/df_actual[x[1]].iloc[0]))
            diffs.append(diff)
            if (diff <= thred):
                count_actual = count_actual + 1
        totalfair_actual = count_actual/36
        
        count_pred = 0
        totalfair_pred = 0
        race_columns = [x for x in df_pred.columns if (x.count("_") == 0) and x != 'total']
        perm_lst, column_name_lst = permutation(df_pred[race_columns])
        for x in perm_lst:
            fem1 = x[0] + '_'+'fem'
            fem2 = x[1] + "_"+'fem'
            if (abs((df_pred[fem1].iloc[0]/df_pred[x[0]].iloc[0]) - (df_pred[fem2].iloc[0]/df_pred[x[1]].iloc[0])) <= thred):
                count_pred = count_pred + 1
        totalfair_pred = count_pred/36
        return totalfair_actual, totalfair_pred, diffs
    
    elif (level == 'l3'):
        diffs = []
        # constrauct a new tree for race & first-gen-status
        race_columns = [x for x in df_actual.columns if (x.count("_") == 0) and x != 'total'] 
        for race in race_columns:
            # r_first = r_male_first + r_fem_first
            col_name_firstgen = race + '_' + 'first'
            col_male_firstgen = race + '_male_first' 
            col_fem_firstgen = race + '_fem_first' 
            df_actual[col_name_firstgen] = df_actual[col_male_firstgen] + df_actual[col_fem_firstgen]
            df_pred[col_name_firstgen] = df_pred[col_male_firstgen] + df_pred[col_fem_firstgen]
            # r_nFirst = r_male_nFirst + r_fem_nFirst
            col_name_nfirstgen = race + '_' + 'nFirst'
            col_male_nfirstgen = race + '_male_nFirst' 
            col_fem_nfirstgen = race + '_fem_nFirst' 
            df_actual[col_name_nfirstgen] = df_actual[col_male_nfirstgen] + df_actual[col_fem_nfirstgen]
            df_pred[col_name_nfirstgen] = df_pred[col_male_nfirstgen] + df_pred[col_fem_nfirstgen]

        ################ to calculate the total fairness ##################
        perm_lst, column_name_lst = permutation(df_actual[race_columns])
        totalfairs_actual = 0
        y1 = 0
        y2 = 0
        y12 = 0
        for x in perm_lst:
            # y1 = r1_fem/r1 - r2_fem/r2
            # y2 = r1_firstgen/r1 - r2_firstgen/r2
            fem1 = x[0] + '_'+'fem'
            fem2 = x[1] + "_"+'fem'
            fg1 = x[0] + '_'+'first'
            fg2 = x[1] + "_"+'first'
            diff1 = abs((df_actual[fem1].iloc[0]/df_actual[x[0]].iloc[0]) - (df_actual[fem2].iloc[0]/df_actual[x[1]].iloc[0]))
            diffs.append(diff1)

            if (diff1 <= thred):
                y1 = 1
            diff2 = abs((df_actual[fg1].iloc[0]/df_actual[x[0]].iloc[0]) - (df_actual[fg2].iloc[0]/df_actual[x[1]].iloc[0]))
            diffs.append(diff2)

            if ( diff2 <= thred):
                y2 = 1
                
            y12 = y1 * y2
            f = 1/3 * y1 + 1/3 * y2 + 1/3 * y12
            totalfairs_actual = totalfairs_actual + f
        
        totalfair_actual = totalfairs_actual/36
        
        ####### for prediction
        totalfairs_pred = 0
        y1 = 0
        y2 = 0
        y12 = 0
        for x in perm_lst:
            fem1 = x[0] + '_'+'fem'
            fem2 = x[1] + "_"+'fem'
            fg1 = x[0] + '_'+'first'
            fg2 = x[1] + "_"+'first'
            if (abs((df_pred[fem1].iloc[0]/df_pred[x[0]].iloc[0]) - (df_pred[fem2].iloc[0]/df_pred[x[1]].iloc[0])) <= thred):
                y1 =  1
            if (abs((df_pred[fg1].iloc[0]/df_pred[x[0]].iloc[0]) - (df_pred[fg2].iloc[0]/df_pred[x[1]].iloc[0])) <= thred):
                y2 = 1
            y12 = y1 * y2
            f = 1/3 * y1 + 1/3 * y2 + 1/3 * y12
            totalfairs_pred = totalfairs_pred + f
        
        totalfair_pred = totalfairs_pred/36
        
        return totalfair_actual, totalfair_pred, diffs

        


In [233]:
directory = os.getcwd() + '\\' + 'data'
files = os.listdir(directory)
dfs = {}
dfs_standard = {}
dfs_sequential = {}
for filename in files: # read all the files
    if filename.endswith('.csv'):
        #concast the file name
        dir_filename = "data/"+filename
        f = pd.read_csv(dir_filename, sep="\t" , encoding="utf-16-le",  thousands=',')        
        dfs[filename] = f
        

## 1. Data cleaning

data cleaning for standard aggreagtion

In [234]:
for name in dfs:

    # get the features of df
    df_name = name
    df_name = df_name.split('.')[0].split('_')
    Student_level = df_name[0]
    First_gen_status = df_name[1]
    Enrollment_status = df_name[2]
    Gender = df_name[3]
    Residency = df_name[4]
    Entry_status = df_name[5]
    
    
    # operation for datasets
    df = dfs[name]
    df = df.drop(df.index[-1])
    df.loc[df['Category'] == 'Chinese', 'Broad category'] = 'East Asain'
    df.loc[df['Category'] == 'Taiwanese', 'Broad category'] = 'East Asain'
    df.loc[df['Category'] == 'Japanese', 'Broad category'] = 'East Asain'
    df.loc[df['Category'] == 'Korean', 'Broad category'] = 'East Asain'
    df.loc[df['Category'] == 'White/Middle Eastern', 'Broad category'] = 'White/Middle Eastern'
    data_2010_columns = ["Broad category", '2010', '2011', '2012', '2013','2014', '2015','2016', '2017', '2018', '2019']
    df = df[data_2010_columns]
    df = df.groupby("Broad category").sum()
    df = df.rename(index = {'American Indian/Alaska Native':'Native', 'African American and Black' : 'Black', 'Hispanic/Latinx': 'Hiapanic', 
                           'Native Hawaiian and Pacific Islander': 'Islander', 'Southwest Asian/North African' : 'SANA', 'White/Middle Eastern':'WE'})

    # transpose
    df = df.T
    df = df.rename_axis(None, axis = 1).rename_axis('year', axis = 0)
    df = df.reset_index()
    # store the dataframes into df_standard    

    # melt the dataset
    columns = df.columns # get all the columns' names
    columns = columns[1:] # get the columns names except for Year

    df = pd.melt(df, id_vars=["year"], value_vars=columns)
    df = df.rename(columns = {0:'Race', 'value':'count'})

    # add columns for other features
    df["Student_Level"] = Student_level
    df["First_gen_status"] = First_gen_status
    df["Enrollment_status"] = Enrollment_status
    df["Gender"] = Gender
    df["Residency"] = Residency
    df["Entry_status"] = Entry_status
    
    dfs_standard[name] = df

##################### merge the datasets ########################################

names = list(dfs_standard.keys())
datasets = list(dfs_standard.values())
# concat all the datasets together
df_standard = pd.concat(datasets, sort=False, ignore_index = True)
df_standard = df_standard.sort_values(by = ['year','Student_Level','Residency','Enrollment_status','Gender'])
df_standard = df_standard.reset_index().drop(['index'], axis = 1)
df_standard = df_standard.rename(columns = {'variable':'race'})
df_standard.columns= df_standard.columns.str.lower()

data cleaning for sequential aggreagtion

In [235]:
directory = os.getcwd() + '\\' + 'data'
files = os.listdir(directory)
dfs = {}
dfs_standard = {}
dfs_sequential = {}
for filename in files: # read all the files
    if filename.endswith('.csv'):
        #concast the file name
        dir_filename = "data/"+filename
        f = pd.read_csv(dir_filename,  sep="\t" , encoding="utf-16-le",  thousands=',')        
        dfs[filename] = f
        

In [236]:
# read the file of sequential aggreation
# for alpha in ['0', '0.2', '0.4','0.6', '0.8', '1']:
#     for epsilon in ['0.05', '0.1', '0.15','0.02', '0.25', '0.03']:

# a 2-D list to store all the categories
rules = []
epsilon = 0

for alpha in ['0.8']:
    for epsilon in ['0.20']:
        rule_name = alpha + " " + epsilon +".txt"
print(epsilon)

file_path = os.getcwd() + '\\' + 'sequential aggregation rules' + '\\' + rule_name

# open the file
rule_file = open(file_path, 'r')
# read every line 
lines = rule_file.readlines()
for line in lines:
    if (line !='\n'):
        line = line.rstrip('\n').split(',')
        rules.append(line)


0.20


In [237]:
# define labels
def label_race (row):
    if row['Category'] in rules[0]:
        return 'G1'
    
    if row['Category'] in rules[1]:
        return 'G2'
    
    if row['Category'] in rules[2] :
        return 'G3'
    
    if row['Category'] in rules[3] :
        return 'G4'
    
    if row['Category'] in rules[4] :
        return 'G5'
    
    if row['Category'] in rules[5]:
        return 'G6'
    
    if row['Category'] in rules[6] :
        return 'G7'
        
    if row['Category'] in rules[7] :
        return 'G8'    
    
    if row['Category'] in rules[8] :
        return 'G9'

In [238]:
for name in dfs:

    # get the features of df
    df_name = name
    df_name = df_name.split('.')[0].split('_')
    Student_level = df_name[0]
    First_gen_status = df_name[1]
    Enrollment_status = df_name[2]
    Gender = df_name[3]
    Residency = df_name[4]
    Entry_status = df_name[5]
    
    
    # operation for datasets
    df = dfs[name]
    # # apply the sequential aggreagtion rules
    df['SA category'] = df.apply(lambda row: label_race(row), axis=1)
    data_2010_columns = ["SA category", '2010', '2011', '2012', '2013','2014', '2015','2016', '2017', '2018', '2019']
    df = df[data_2010_columns]
    df = df.groupby("SA category").sum()
#     # transpose
    df = df.T
    df = df.rename_axis(None, axis = 1).rename_axis('year', axis = 0)
    df = df.reset_index()
    # store the dataframes into df_standard    

    # melt the dataset
    columns = df.columns # get all the columns' names
    columns = columns[1:] # get the columns names except for Year

    df = pd.melt(df, id_vars=["year"], value_vars=columns)
    df = df.rename(columns = {0:'Race', 'value':'count'})

    # add columns for other features
    df["Student_Level"] = Student_level
    df["First_gen_status"] = First_gen_status
    df["Enrollment_status"] = Enrollment_status
    df["Gender"] = Gender
    df["Residency"] = Residency
    df["Entry_status"] = Entry_status
    
    dfs_sequential[name] = df

##################### merge the datasets ########################################

names = list(dfs_sequential.keys())
datasets = list(dfs_sequential.values())
# concat all the datasets together
df_sequential = pd.concat(datasets, sort=False, ignore_index = True)
df_sequential = df_sequential.sort_values(by = ['year','Student_Level','Residency','Enrollment_status','Gender'])
df_sequential = df_sequential.reset_index().drop(['index'], axis = 1)
df_sequential = df_sequential.rename(columns = {'variable':'race'})
df_sequential.columns= df_sequential.columns.str.lower()

## 2. Forecast

In [239]:
#####
# to perform the prediction task
# parameters: input df
# return: total rmse and the predicted result (one row)
#####
def pred(df, period):
    pred_results = []
    df_vars = df.columns 
    rmse = []
    for cl in df_vars:
        # fit the model for every column 
        # size of training set: 21, size of test set: 1
        
        train = df[cl].iloc[0:-period]
        test = df[cl].iloc[-period:]
        # fit the model
        # model = auto_arima(train)
        model = SimpleExpSmoothing(train)
        fit = model.fit()
        pred = fit.forecast(period)
        # pred = model.predict(n_periods=period)
        # convert the pred series to dataframe
        df_pred = pd.DataFrame(pred)
        df_pred = df_pred.rename({0:cl}, axis = 1)
        # attach the pred to the list
        pred_results.append(df_pred)
        # calculate RMSE
        RMSE = sqrt(mean_squared_error(test, pred))
        rmse.append(RMSE)
    pred = pd.concat(pred_results, axis=1)
    total_rmse = sum(rmse)/len(rmse)
    
    return (pred, total_rmse)

def get_proportion_AHP(df, feature_name):

    # find the forecast results for residency

    ### calculate the porpotion of each residency w.r.t total amount
    # get categories of residency
    feature_lst = list(df[feature_name].unique())
    # list to stor mean proportions for every subcategory in the feature
    mean_ppt_feature_lst = []
    # dataframe for different features
    df_new = df.pivot_table(index= "year", columns = [feature_name], values = "count", aggfunc= "sum")
    # create a sum column 
    df_new['total'] = df_new.sum(axis = 1)
    # get proportion for every race in total trips
    for i in feature_lst:
        sum_ppt = 0
        # for every year
        for j in range(0, len(df_new)):
            # proportion of the race (race/total) in every year
            ppt = df_new[i].iloc[j]/df_new['total'].iloc[j]
            # add all proportions
            sum_ppt = sum_ppt + ppt
        # get the mean of the sum
        mean_ppt = sum_ppt/len(df_new)
        mean_ppt_feature_lst.append(mean_ppt)
    feature_ppt = dict(zip(feature_lst, mean_ppt_feature_lst))

    return feature_ppt

def get_proportion_PHA (df, feature_name):

    # find the forecast results for residency

    ### calculate the porpotion of each residency w.r.t total amount
    # get categories of residency
    feature_lst = list(df[feature_name].unique())
    # list to stor mean proportions for every race
    mean_ppt_feature_lst = []
    # dataframe for races
    df_new = df.pivot_table(index= "year", columns = [feature_name], values = "count", aggfunc= "sum")
    df_new['total'] = df_new.sum(axis = 1)
    # get proportion for every race in total trips
    for i in feature_lst:
        mean_ppt = df_new[i].mean()/df_new['total'].mean()
        mean_ppt_feature_lst.append(mean_ppt)
    feature_ppt = dict(zip(feature_lst, mean_ppt_feature_lst))

    return feature_ppt

standard aggreagtion & sequential aggregation

In [240]:
def forecasts (df, proportion):
    df.columns= df.columns.str.lower()
    # aggregate enrollment_status and entry_status and residency together
    ###################################################################

    ## to seperate it into 3 levels
    df_std_l3 = df.pivot_table(index= "year", columns = ["race", "gender", "first_gen_status"], values = "count", aggfunc= "sum")
    df_std_l3.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_std_l3.columns]

    ## 2 levels: total -> race -> gender
    df_std_l2 = df.pivot_table(index= "year", columns = ["race", "gender"], values = "count", aggfunc= "sum")
    df_std_l2.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_std_l2.columns]

    ## 1 level: total -> first_gen_status
    df_std_l1 = df.pivot_table(index= "year", columns = ["race"], values = "count", aggfunc= "sum")

    ## total
    df_total = df.pivot_table(index= "year", values = "count", aggfunc= "sum")
    df_total = df_total.rename(columns={"count": "total"})

    # join the dateframes
    df_std_full_info = df_total.join(df_std_l1).join(df_std_l2).join(df_std_l3)

    # change the index year to day-time format
    df_std_full_info.index = pd.to_datetime(df_std_full_info.index, format='%Y')
    df_std_full_info = df_std_full_info.to_period("Y")
    df_std_full_info = df_std_full_info.to_timestamp()
    df_std_full_info
    df_std_full_info.head(10)

    ################################### prediction for data without info loss #########################
    pred_df_std_full_info, pred_df_std_full_info_rmse = pred(df_std_full_info, 1)
    pred_df_std_full_info = pd.concat([df_std_full_info,pred_df_std_full_info])

    ################################### prediction for data with info loss at level 3 #########################
    df_std_loss_l3 = df_total.join(df_std_l1).join(df_std_l2)
    df_std_loss_l3.index = pd.to_datetime(df_std_loss_l3.index, format='%Y')
    df_std_loss_l3 = df_std_loss_l3.to_period("Y")
    df_std_loss_l3 = df_std_loss_l3.to_timestamp()
    pred_df_std_loss_l3, pred_df_std_loss_l3_rmse = pred(df_std_loss_l3,1)

    # create the forecast results for every residency using AHP
    if (proportion == 'AHP'):
        fg_ppt_AHP = get_proportion_AHP(df, "first_gen_status")
    elif (proportion == 'PHA'):
        fg_ppt_AHP = get_proportion_PHA(df, "first_gen_status")
    # create the forecast results for every residency using AHP
    for col in pred_df_std_loss_l3:
        if col.count('_') == 1:
            gender = col.split("_")[-1]
            # find all the first_gen_status for the state
            for fg in list(df['first_gen_status'].unique()):
                new_col_name = col + "_" + fg
                pred_df_std_loss_l3[new_col_name] = pred_df_std_loss_l3[col]*fg_ppt_AHP[fg]
    pred_df_std_loss_l3 = pd.concat([df_std_full_info,pred_df_std_loss_l3])
    
    ################################### prediction for data with info loss at level 2 #########################

    # join the dateframes without race and residency
    df_std_loss_l2 = df_total.join(df_std_l1)
    df_std_loss_l2.index = pd.to_datetime(df_std_loss_l2.index, format='%Y')
    df_std_loss_l2 = df_std_loss_l2.to_period("Y")
    df_std_loss_l2 = df_std_loss_l2.to_timestamp()
    pred_df_std_loss_l2, pred_df_std_loss_l2_rmse = pred(df_std_loss_l2,1)
    
    if (proportion == 'AHP'):
        gender_ppt = get_proportion_AHP(df, "gender")
    elif (proportion == 'PHA'):
        gender_ppt = get_proportion_PHA(df, "gender")
        
    # create the forecast results for every gender using AHP
    for col in pred_df_std_loss_l2:
        if col != 'total':
            race = col
            # find all the genders for the race
            for gen in list(df['gender'].unique()):
                new_col_name = col + "_" + gen
                pred_df_std_loss_l2[new_col_name] = pred_df_std_loss_l2[col]*gender_ppt[gen]

    if (proportion == 'AHP'):
        fg_ppt = get_proportion_AHP(df, "first_gen_status")
    elif (proportion == 'PHA'):
        fg_ppt = get_proportion_PHA(df, "first_gen_status")
        
    for col in pred_df_std_loss_l2:
        if col.count('_') == 1:
            gender = col.split("_")[-1]
            # find all the regions for the state
            for fg in list(df['first_gen_status'].unique()):
                new_col_name = col + "_" + fg
                pred_df_std_loss_l2[new_col_name] = pred_df_std_loss_l2[col]*fg_ppt[fg]

    # attach the row to the original dataframe as the last row
    pred_df_std_loss_l2 = pd.concat([df_std_full_info,pred_df_std_loss_l2])
    pred_df_std_loss_l2
    # output the csv result
    
    
    # return all the results
    return pred_df_std_full_info, pred_df_std_loss_l3, pred_df_std_loss_l2

In [241]:
########################### standard aggregation

## AHP
AHP_pred_df_std_full_info, AHP_pred_df_std_loss_l3, AHP_pred_df_std_loss_l2 = forecasts(df_standard, "AHP")
PHA_pred_df_std_full_info, PHA_pred_df_std_loss_l3, PHA_pred_df_std_loss_l2 = forecasts(df_standard, "PHA")

## PHA
AHP_pred_df_seq_full_info, AHP_pred_df_seq_loss_l3, AHP_pred_df_seq_loss_l2 = forecasts(df_sequential, "AHP")
PHA_pred_df_seq_full_info, PHA_pred_df_seq_loss_l3, PHA_pred_df_seq_loss_l2 = forecasts(df_sequential, "PHA")

In [242]:
AHP_pred_df_seq_full_info

,total,G1,G2,G3,G4,G5,G6,G7,G8,G9,...,G7_male_first,G7_male_nFirst,G8_fem_first,G8_fem_nFirst,G8_male_first,G8_male_nFirst,G9_fem_first,G9_fem_nFirst,G9_male_first,G9_male_nFirst
2010-01-01,30723.0,964.0,4971.0,966.0,17672.0,760.0,101.0,3572.000000,148.0,1569.0,...,984.000000,631.0,62.0,24.0,38.0,24.0,356.0,467.0,304.0,442.0
2011-01-01,33939.0,1026.0,5472.0,1545.0,18876.0,935.0,161.0,3746.000000,213.0,1965.0,...,964.000000,696.0,93.0,39.0,49.0,32.0,499.0,593.0,362.0,511.0
2012-01-01,36940.0,1126.0,6093.0,1898.0,20375.0,1153.0,188.0,3730.000000,246.0,2131.0,...,992.000000,742.0,113.0,40.0,52.0,41.0,563.0,641.0,373.0,554.0
2013-01-01,39328.0,1185.0,6624.0,2350.0,21357.0,1270.0,230.0,3631.000000,294.0,2387.0,...,893.000000,711.0,109.0,59.0,76.0,50.0,662.0,742.0,419.0,564.0
2014-01-01,41884.0,1281.0,7475.0,2817.0,21450.0,1695.0,285.0,3781.000000,371.0,2729.0,...,907.000000,742.0,134.0,84.0,93.0,60.0,785.0,838.0,515.0,591.0
2015-01-01,42233.0,1292.0,7849.0,2920.0,21337.0,1749.0,314.0,3556.000000,383.0,2833.0,...,818.000000,700.0,132.0,97.0,80.0,74.0,824.0,834.0,515.0,660.0
2016-01-01,45007.0,1413.0,9033.0,3257.0,21653.0,1861.0,364.0,3640.000000,433.0,3353.0,...,810.000000,754.0,142.0,129.0,72.0,90.0,1091.0,923.0,674.0,665.0
2017-01-01,45914.0,1447.0,9639.0,3354.0,21723.0,1838.0,400.0,3483.000000,462.0,3568.0,...,707.000000,778.0,149.0,135.0,69.0,109.0,1252.0,940.0,728.0,648.0
2018-01-01,46903.0,1505.0,10098.0,3425.0,21945.0,1806.0,402.0,3624.000000,435.0,3663.0,...,692.000000,854.0,146.0,127.0,72.0,90.0,1373.0,943.0,712.0,635.0
2019-01-01,47269.0,1515.0,10552.0,3676.0,21362.0,1862.0,346.0,3678.000000,521.0,3757.0,...,661.000000,876.0,159.0,158.0,95.0,109.0,1408.0,1007.0,684.0,658.0


# Evaluation

In [243]:
character_datasets = {
    
    
            'AHP_pred_df_std_full_info':AHP_pred_df_std_full_info,
            'AHP_pred_df_seq_full_info': AHP_pred_df_seq_full_info,
            'PHA_pred_df_std_full_info':PHA_pred_df_std_full_info,
            'PHA_pred_df_seq_full_info':PHA_pred_df_seq_full_info,
            
            
            'AHP_pred_df_std_loss_l3': AHP_pred_df_std_loss_l3,
            'PHA_pred_df_std_loss_l3':PHA_pred_df_std_loss_l3,
            
            'AHP_pred_df_seq_loss_l3': AHP_pred_df_seq_loss_l3,
            'PHA_pred_df_seq_loss_l3':PHA_pred_df_seq_loss_l3,
            
            'AHP_pred_df_std_loss_l2': AHP_pred_df_std_loss_l2,
            'PHA_pred_df_std_loss_l2': PHA_pred_df_std_loss_l2,
            
            'AHP_pred_df_seq_loss_l2': AHP_pred_df_seq_loss_l2,
            'PHA_pred_df_seq_loss_l2': PHA_pred_df_seq_loss_l2,
            
            }

################### RMSE ########################

# get the rmse for different level...
character_datasets_rmse = {}
character_datasets_rmse_total = {}
for x in character_datasets:
    # bottom level
    bottom_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 2]
    # second level
    sec_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 1]
    # first level
    fst_level_cols = [col for col in character_datasets[x].columns if (col.count("_") == 0 and col != "total")]
    # total
    total = ['total']
    levels = [bottom_level_cols,sec_level_cols, fst_level_cols, total]
    levels_name = ['bottom_level_cols','sec_level_cols', 'fst_level_cols', 'total']
    for i in range(0, len(levels)):
        df_actual = character_datasets[x][levels[i]].iloc[-2:-1]
        df_pred = character_datasets[x][levels[i]].iloc[-1:]
        rmse_df, total_rmse, avg_rmse, num_of_cols = RMSE(df_actual,df_pred)
        key = x + " " + levels_name[i]
        
        character_datasets_rmse[key] = [rmse_df, total_rmse, avg_rmse, num_of_cols]
        character_datasets_rmse_total[key] = [total_rmse, avg_rmse, num_of_cols]
        
# get the MASE for different level
character_datasets_mase = {}
character_datasets_mase_total = {}
for x in character_datasets:
    # bottom level
    bottom_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 2]
    # second level
    sec_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 1]
    # first level
    fst_level_cols = [col for col in character_datasets[x].columns if (col.count("_") == 0 and col != "total")]
    # total
    total = ['total']
    levels = [bottom_level_cols,sec_level_cols, fst_level_cols, total]
    levels_name = ['bottom_level_cols','sec_level_cols', 'fst_level_cols', 'total']
    for i in range(0, len(levels)):
        df_actual = character_datasets[x][levels[i]].iloc[-2:-1]
        df_pred = character_datasets[x][levels[i]].iloc[-1:]
        df_naive = character_datasets[x][levels[i]].iloc[:-1]
        mase_df, total_mase, avg_mase, num_of_cols = MASE(df_actual,df_pred,df_naive)
        key = x + " " + levels_name[i]
        character_datasets_mase[key] = [mase_df, total_mase, avg_mase, num_of_cols]
        character_datasets_mase_total[key] = [total_mase, avg_mase, num_of_cols]
        


In [244]:
### calculate total fairness
character_datasets_fairness = {}


for x in character_datasets:
    # bottom level
    bottom_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 2]
    # second level
    sec_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 1]
    # first level
    fst_level_cols = [col for col in character_datasets[x].columns if (col.count("_") == 0 and col != "total")]
    # cols
    cols = bottom_level_cols + sec_level_cols + fst_level_cols
    levels = ["l3", 'l2', 'l1']
    for i in range(0, len(levels)):
        # get the mean of all the time periods 
        # drop the total 
        df_actual = pd.DataFrame(character_datasets[x].mean(axis = 0)).T
        df_pred = character_datasets[x].iloc[-1:]
        
        level = levels[i]
        epsilon = 0.01
        totalfair_actual, totalfair_pred, diff = total_fairness(df_actual,df_pred,float(epsilon),level)
        
        key = x + " " + levels_name[i]
        character_datasets_fairness[key] = [totalfair_actual, totalfair_pred]


In [245]:
# ################## result for standard aggregation
# file_path = os.getcwd() + '\\' + 'results' + '\\' + 'accuracy_standard ' + rule_name
# with open(file_path, 'w+') as f:
#     print('RMSE', file = f)
#     print('All info aval (most accurate)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_full_info bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_full_info sec_level_cols'][2]), file = f)
#     print('    level 1', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_full_info fst_level_cols'][2]), file = f)
#     print('    total', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_full_info total'][2]), file = f)   
#     print()
    
#     print('MASE', file = f)
#     print('All info aval (most accurate)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_full_info bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_full_info sec_level_cols'][2]), file = f)
#     print('    level 1', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_full_info fst_level_cols'][2]), file = f)
#     print('    total', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_full_info total'][2]), file = f)   
#     print('\n', file = f)
    
    
    
    
    
#     print('RMSE', file = f)
#     print('loss info at level 3 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_loss_l3 bottom_level_cols'][2]), file = f)

#     print('RMSE', file = f)
#     print('loss info at level 3 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['PHA_pred_df_std_loss_l3 bottom_level_cols'][2]), file = f)
    
#     print('MASE', file = f)
#     print('loss info at level 3 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_loss_l3 bottom_level_cols'][2]), file = f)

#     print('MASE', file = f)
#     print('loss info at level 3 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['PHA_pred_df_std_loss_l3 bottom_level_cols'][2]), file = f)
    
#     print('\n', file = f)

    
    
#     print('RMSE', file = f)
#     print('loss info at level 2 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_loss_l2 sec_level_cols'][2]), file = f)
#     print('MASE', file = f)
#     print('loss info at level 2 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_std_loss_l2 sec_level_cols'][2]), file = f)
    

#     print('RMSE', file = f)
#     print('loss info at level 2 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['PHA_pred_df_std_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_std_loss_l2 sec_level_cols'][2]), file = f)
    

#     print('MASE', file = f)
#     print('loss info at level 2 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['PHA_pred_df_std_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_mase['PHA_pred_df_std_loss_l2 sec_level_cols'][2]), file = f)



In [246]:
# #result for sequential aggregation

# file_path = os.getcwd() + '\\' + 'results' + '\\' + 'accuracy ' + rule_name

# with open(file_path, 'w+') as f:
#     print('RMSE', file = f)
#     print('All info aval (most accurate)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_full_info bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_full_info sec_level_cols'][2]), file = f)
#     print('    level 1', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_full_info fst_level_cols'][2]), file = f)
#     print('    total', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_full_info total'][2]), file = f)   
#     print()
    
#     print('MASE', file = f)
#     print('All info aval (most accurate)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_full_info bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_full_info sec_level_cols'][2]), file = f)
#     print('    level 1', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_full_info fst_level_cols'][2]), file = f)
#     print('    total', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_full_info total'][2]), file = f)   
#     print('\n', file = f)
    
    
    
    
    
#     print('RMSE', file = f)
#     print('loss info at level 3 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_loss_l3 bottom_level_cols'][2]), file = f)

#     print('RMSE', file = f)
#     print('loss info at level 3 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['PHA_pred_df_seq_loss_l3 bottom_level_cols'][2]), file = f)
    
#     print('MASE', file = f)
#     print('loss info at level 3 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_loss_l3 bottom_level_cols'][2]), file = f)

#     print('MASE', file = f)
#     print('loss info at level 3 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['PHA_pred_df_seq_loss_l3 bottom_level_cols'][2]), file = f)
    
#     print('\n', file = f)

    
    
#     print('RMSE', file = f)
#     print('loss info at level 2 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_loss_l2 sec_level_cols'][2]), file = f)

#     print('MASE', file = f)
#     print('loss info at level 2 (AHP)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_mase['AHP_pred_df_seq_loss_l2 sec_level_cols'][2]), file = f)
    

#     print('RMSE', file = f)
#     print('loss info at level 2 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_rmse['PHA_pred_df_seq_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_rmse['AHP_pred_df_seq_loss_l2 sec_level_cols'][2]), file = f)
    
#     print('MASE', file = f)
#     print('loss info at level 2 (PHA)', file = f)
#     print('    level 3', file = f)
#     print('      ' + str(character_datasets_mase['PHA_pred_df_seq_loss_l2 bottom_level_cols'][2]), file = f)
#     print('    level 2', file = f)
#     print('      ' + str(character_datasets_mase['PHA_pred_df_seq_loss_l2 sec_level_cols'][2]), file = f)



In [247]:

################### result for standard aggregation
file_path = os.getcwd() + '\\' + 'results' + '\\' + 'fairness_standard_fixed ' + rule_name
with open(file_path, 'w+') as f:
    print('fairness_actual, pred, diff', file = f)
    print('All info aval (most accurate)', file = f)
    print('    level 3', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][1] - character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][0]), file = f)
    
    
    print('    level 2', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info sec_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info sec_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info sec_level_cols'][1] - character_datasets_fairness['PHA_pred_df_std_full_info sec_level_cols'][0]), file = f)

    print('    level 1', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info fst_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info fst_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info fst_level_cols'][1] - character_datasets_fairness['PHA_pred_df_std_full_info fst_level_cols'][0]), file = f)


    print('\n', file = f)
    

    print('\n', file = f)
    


    print('fairness_acutal, pred, diff', file = f)
    print('loss info at level 3', file = f)
    print('    level 3', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][0]), file = f)

    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_loss_l3 bottom_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_loss_l3 bottom_level_cols'][1] - character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][0]), file = f)

    print('\n', file = f)
    

    print('\n', file = f)
    
    print('fairness_actual, pred, diff ', file = f)
    print('loss info at level 2', file = f)
    print('    level 3', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_loss_l2 bottom_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_loss_l2 bottom_level_cols'][1] - character_datasets_fairness['PHA_pred_df_std_full_info bottom_level_cols'][0]), file = f)

    print('    level 2', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_full_info sec_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_loss_l2 sec_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_std_loss_l2 sec_level_cols'][1] - character_datasets_fairness['PHA_pred_df_std_full_info sec_level_cols'][0]), file = f)
    
    




In [248]:

################### result for sequential aggregation
file_path = os.getcwd() + '\\' + 'results' + '\\' + 'fairness_fixed ' + rule_name
with open(file_path, 'w+') as f:
    print('fairness_actual, pred, diff', file = f)
    print('All info aval (most accurate)', file = f)
    print('    level 3', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][1] - character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][0]), file = f)
    
    
    print('    level 2', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info sec_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info sec_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info sec_level_cols'][1] - character_datasets_fairness['PHA_pred_df_seq_full_info sec_level_cols'][0]), file = f)

    print('    level 1', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info fst_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info fst_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info fst_level_cols'][1] - character_datasets_fairness['PHA_pred_df_seq_full_info fst_level_cols'][0]), file = f)


    print('\n', file = f)
    

    print('\n', file = f)
    


    print('fairness_acutal, pred, diff', file = f)
    print('loss info at level 3', file = f)
    print('    level 3', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][0]), file = f)

    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_loss_l3 bottom_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_loss_l3 bottom_level_cols'][1] - character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][0]), file = f)

    print('\n', file = f)
    

    print('\n', file = f)
    
    print('fairness_actual, pred, diff ', file = f)
    print('loss info at level 2', file = f)
    print('    level 3', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_loss_l2 bottom_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_loss_l2 bottom_level_cols'][1] - character_datasets_fairness['PHA_pred_df_seq_full_info bottom_level_cols'][0]), file = f)

    print('    level 2', file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_full_info sec_level_cols'][0]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_loss_l2 sec_level_cols'][1]), file = f)
    print('      ' + str(character_datasets_fairness['PHA_pred_df_seq_loss_l2 sec_level_cols'][1] - character_datasets_fairness['PHA_pred_df_seq_full_info sec_level_cols'][0]), file = f)
    
    



In [249]:

# directory = os.getcwd() + '\\' + 'data'
# files = os.listdir(directory)
# dfs = {}
# dfs_standard = {}
# dfs_sequential = {}
# for filename in files: # read all the files
#     if filename.endswith('.csv'):
#         #concast the file name
#         dir_filename = "data/"+filename
#         f = pd.read_csv(dir_filename, sep="\t" , encoding="utf-16-le",  thousands=',')        
#         dfs[filename] = f


# for name in dfs:

#     # get the features of df
#     df_name = name
#     df_name = df_name.split('.')[0].split('_')
#     Student_level = df_name[0]
#     First_gen_status = df_name[1]
#     Enrollment_status = df_name[2]
#     Gender = df_name[3]
#     Residency = df_name[4]
#     Entry_status = df_name[5]
    
    
#     # operation for datasets
#     df = dfs[name]
#     df = df.drop(df.index[-1])
#     data_2010_columns = ["Category", '2010', '2011', '2012', '2013','2014', '2015','2016', '2017', '2018', '2019']
#     df = df[data_2010_columns]
#     df = df.groupby("Category").sum()
   
#     # transpose
#     df = df.T
#     df = df.rename_axis(None, axis = 1).rename_axis('year', axis = 0)
#     df = df.reset_index()
#     # store the dataframes into df_standard    

#     # melt the dataset
#     columns = df.columns # get all the columns' names
#     columns = columns[1:] # get the columns names except for Year

#     df = pd.melt(df, id_vars=["year"], value_vars=columns)
#     df = df.rename(columns = {0:'Race', 'value':'count'})

#     # add columns for other features
#     df["Student_Level"] = Student_level
#     df["First_gen_status"] = First_gen_status
#     df["Enrollment_status"] = Enrollment_status
#     df["Gender"] = Gender
#     df["Residency"] = Residency
#     df["Entry_status"] = Entry_status
    
#     dfs_standard[name] = df

# ##################### merge the datasets ########################################

# names = list(dfs_standard.keys())
# datasets = list(dfs_standard.values())
# # concat all the datasets together
# df_standard = pd.concat(datasets, sort=False, ignore_index = True)
# df_standard = df_standard.sort_values(by = ['year','Student_Level','Residency','Enrollment_status','Gender'])
# df_standard = df_standard.reset_index().drop(['index'], axis = 1)
# df_standard = df_standard.rename(columns = {'variable':'race'})
# df_standard.columns= df_standard.columns.str.lower()

In [250]:
# df = df_standard

# ## 1 level: total -> first_gen_status
# df_std_l3 = df.pivot_table(index= "year", columns = ["race", "gender", "first_gen_status"], values = "count", aggfunc= "sum")
# df_std_l3.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_std_l3.columns]

# ## 2 levels: total -> race -> gender
# df_std_l2 = df.pivot_table(index= "year", columns = ["race", "gender"], values = "count", aggfunc= "sum")
# df_std_l2.columns = ['_'.join(str(s).strip() for s in col if s) for col in df_std_l2.columns]

# ## 1 level: total -> first_gen_status
# df_std_l1 = df.pivot_table(index= "year", columns = ["race"], values = "count", aggfunc= "sum")

# ## total
# df_total = df.pivot_table(index= "year", values = "count", aggfunc= "sum")
# df_total = df_total.rename(columns={"count": "total"})

# # join the dateframes
# df_std_full_info = df_total.join(df_std_l1).join(df_std_l2).join(df_std_l3)

# # change the index year to day-time format
# df_std_full_info.index = pd.to_datetime(df_std_full_info.index, format='%Y')
# df_std_full_info = df_std_full_info.to_period("Y")
# df_std_full_info = df_std_full_info.to_timestamp()
# df_std_full_info
# df_std_full_info.head(10)
# df_std_full_info['Azerbaijani_fem_first'] = 0

# df_std_full_info['Bahraini_male_first'] = 0
# 'Djiboutian_male_first'
# df_std_full_info['Djiboutian_male_first'] = 0
# df_std_full_info['Djiboutian_fem_first'] = 0
# df_std_full_info['Djiboutian_male_nFirst'] = 0
# df_std_full_info['Emerati_male_first'] = 0

# df_std_full_info['Emerati_male_first'] = 0
# df_std_full_info['Georgian_male_first'] = 0
# df_std_full_info['Libyan_fem_first'] = 0
# df_std_full_info['Mauritanian_male_first'] = 0
# df_std_full_info['Mauritanian_male_nFirst'] = 0
# df_std_full_info['Native Hawaiian and Pacific Islander_male_first'] = 0
# df_std_full_info['Native Hawaiian and Pacific Islander_fem_first'] = 0
# df_std_full_info['Native Hawaiian and Pacific Islander_male_nFirst'] = 0
# df_std_full_info['Omani_fem_first'] = 0
# df_std_full_info['Omani_fem_nFirst'] = 0
# df_std_full_info['Other North African_fem_first'] = 0
# df_std_full_info['Qatari_male_first'] = 0
# df_std_full_info['Omani_fem_first'] = 0
# df_std_full_info['Qatari_fem_first'] = 0
# df_std_full_info['Qatari_fem_nFirst'] = 0
# df_std_full_info['Tunisian_fem_first'] = 0
# df_std_full_info['Omani_fem'] = 0
# df_std_full_info['Qatari_fem'] = 0
# df_std_full_info['Tunisian_fem_first'] = 0
# df_std_full_info['Tunisian_fem_first'] = 0
# df_std_full_info['Tunisian_fem_first'] = 0



# character_datasets = {
    
    
#             'AHP_pred_df_std_full_info':df_std_full_info

#             }

# ##################### To test what epsilon to use 
# for x in character_datasets:
#     diffs = {}
#     # bottom level
#     bottom_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 2]
#     # second level

#     cols = bottom_level_cols
#     levels = ["l3"]
#     for i in range(0, len(levels)):
#         # get the mean of all the time periods 
#         # drop the total 
#         df_actual = pd.DataFrame(character_datasets[x].mean(axis = 0)).T

#         level = levels[i]
    
#         totalfair_actual, totalfair_pred, diff = total_fairness(df_actual,df_actual,0.02,level)
#         diffs[level] = diff

       

# plt.figure(figsize=(10, 8), dpi=80)

# plt.plot(diffs['l3'], color='pink', marker='.', ls = '') #plot the data

# X = [0, 40, 5500]
# Y1 = [0.02, 0.02, 0.02]
# Y2 = [0.10, 0.10, 0.10]
# Y3= [0.20, 0.20, 0.20]
# Y4= [0.40, 0.40, 0.40]
# plt.axis('off')
# plt.plot(X,Y1, label = 'epsilon = 0.01')
# plt.plot(X,Y2, label = 'epsilon = 0.05')
# plt.plot(X,Y3, label = 'epsilon = 0.10')
# plt.plot(X,Y4, label = 'epsilon = 0.20')
# plt.legend()


In [251]:
# ###################### To test what epsilon to use 
# character_actual_datasets = {
    
    
#             'AHP_pred_df_std_full_info':AHP_pred_df_std_full_info,

#             }
# for x in character_actual_datasets:
#     diffs = []
#     # bottom level
#     bottom_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 2]
#     # second level
#     sec_level_cols = [col for col in character_datasets[x].columns if col.count("_") == 1]
#     # first level
#     fst_level_cols = [col for col in character_datasets[x].columns if (col.count("_") == 0 and col != "total")]
#     # cols
#     cols = bottom_level_cols + sec_level_cols + fst_level_cols
#     levels = ["l3", 'l2', 'l1']
#     for i in range(0, len(levels)):
#         # get the mean of all the time periods 
#         # drop the total 
#         df_actual = pd.DataFrame(character_datasets[x].mean(axis = 0)).T
#         df_pred = character_datasets[x].iloc[-1:]
        
#         level = levels[i]
#         totalfair_actual, totalfair_pred, diff = total_fairness(df_actual,df_pred,0.02,level)
#         diffs.append(diff)
#         key = x + " " + levels_name[i]
#         character_datasets_fairness[key] = [totalfair_actual, totalfair_pred]

# plt.plot(diffs[0], color='magenta', marker='D', ls = '', label = 'level 3') #plot the data
# plt.plot(diffs[1], color='magenta', marker='D', ls = '', label = 'level 2') #plot the data
# plt.plot(diffs[2], color='magenta', marker='D', ls = '', label = 'level 1')
# plt.plot(diffs[2], color='magenta', marker='D', ls = '', label = 'level 1')
# X = [0, 40, 75]
# Y1 = [0.02, 0.02, 0.02]
# Y2 = [0.05, 0.05, 0.05]
# Y3= [0.10, 0.10, 0.10]
# Y4= [0.20, 0.20, 0.20]

# plt.plot(X,Y1)
# plt.plot(X,Y2)
# plt.plot(X,Y3)
# plt.plot(X,Y4)


In [252]:
# l3_x_acc = [0.76, 0.72, 0.73, 0.73, 0.79, 0.79, 0.81, 0.76, 0.88, 0.89, 0.82, 0.79, 0.83, 0.70, 0.76, 0.82, 0.99, 0.83, 0.77, 0.78, 0.73]
# l3_y_fair_acc = [0.009, 0.176 , 0.055, 0.139, 0.000, 0.018, 0.028, 0.000, 0.315, 0.583, 0.315, 0.352, 0.361, 0.000, 0.185, 0.213, 0.537, 0.019, 0.593, 0.055, 0.055]
# l3_y_fairness = [0.537, 0.352, 0.630, 0.676, 0.565, 0.852, 0.324, 0.324 ,0.278, 0.889, 0.639, 0.861, 0.639, 0.537, 0.500, 0.315, 0.814, 0.843, 0.333, 0.750 ,0.028]
# plt.scatter(l3_x_acc,l3_y_fairness, label = 'Level 3')

# l2_x_acc = [0.76, 0.64, 0.59, 0.67, 0.73 , 0.72, 0.70, 0.70, 0.79, 0.86, 0.65, 0.85, 0.82, 0.78, 0.54, 0.79, 0.85, 0.70, 0.61, 0.59, 0.58]
# l2_y_fairness = [0.222, 0.194 , 0.139, 0.139, 0.111, 0.194, 0.167, 0.194, 0.167, 0.139, 0.278, 0.139, 0.139, 0.139, 0.750, 0.167 ,0.167, 0.111, 0.222, 0.139, 0.139]
# l2_y_fair_acc = [0.000, 0.111, 0.028, 0.083, 0.083, 0.111, 0.056, 0.055, 0.000, 0.028, 0.111, 0.028, 0.083, 0.083, 0.083, 0.084, 0.000, 0.028, 0.056, 0.056, 0.028]
# plt.scatter(l2_x_acc,l2_y_fairness, label = 'Level 2')


# l1_x_acc = [0.64, 0.49, 0.57, 0.65, 0.67, 0.61, 0.63, 0.52, 0.77, 0.75, 0.60, 0.77, 0.80, 0.65, 0.39, 0.74, 0.79, 0.63, 0.49, 0.52, 0.50]
# l1_y_fairness = [0.250,0.139, 0.111, 0.139, 0.083, 0.111, 0.056, 0.083, 0.167, 0.111, 0.056, 0.111, 0.056, 0.056, 0.333, 0.222, 0.083, 0.167, 0.222, 0.083, 0.028]
# l1_y_fair_acc = [0.083, 0.000, 0.000, 0.056, 0.027, 0.000, 0.028, 0.000, 0.056, 0.000, 0.028, 0.000, 0.027, 0.000, 0.028, 0.028, 0.056, 0.000, 0.055, 0.028, 0.055]

# plt.scatter(l1_x_acc,l1_y_fairness, label = 'Level 1')

# plt.xlabel("MASE")
# plt.ylabel("Total fairness")
# plt.legend(prop={'size': 15})

In [253]:
# plt.scatter(l3_x_acc,l3_y_fair_acc, label = 'Level 3')
# plt.scatter(l2_x_acc,l2_y_fair_acc, label = 'Level 2')
# plt.scatter(l1_x_acc,l1_y_fair_acc, label = 'Level 1')
# plt.xlabel("Firness ")
# plt.ylabel("Total fairness")
# plt.legend(prop={'size': 15})